In [68]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

In [69]:
def process_wind_file(filepath):
    df = pd.read_table(filepath, delim_whitespace=True)
    df = df.drop(index=df.index[0])
    df = df[df['#YY'] == filepath]

    for col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    #print(df)
    df = df.groupby(['MM','DD', '#YY']).mean().reset_index()
    df.rename(columns={'#YY': 'year', 'MM': 'month', 'DD': 'day'}, inplace=True)
    #add time to df which has datetime
    df['time'] = pd.to_datetime(df[['year', 'month', 'day']])
    df['time'] = df['time'].dt.date
    return df[['time','WSPD']]

process_wind_file("Data/Wind/wind2009.txt")

/var/folders/5y/74w_zv855875xvjtmhp4g1xw0000gn/T/ipykernel_22347/1717233576.py:2: FutureWarning: The 'delim_whitespace' keyword in pd.read_table is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_table(filepath, delim_whitespace=True)


,time,WSPD


In [70]:
fin_df = process_wind_file("Data/Wind/wind2008.txt")
for i in range(2009, 2024, 1):
    #print(i)
    fin_df = pd.concat([fin_df, process_wind_file("Data/Wind/wind" + str(i) + ".txt")])
fin_df

/var/folders/5y/74w_zv855875xvjtmhp4g1xw0000gn/T/ipykernel_22347/1717233576.py:2: FutureWarning: The 'delim_whitespace' keyword in pd.read_table is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_table(filepath, delim_whitespace=True)
/var/folders/5y/74w_zv855875xvjtmhp4g1xw0000gn/T/ipykernel_22347/1717233576.py:2: FutureWarning: The 'delim_whitespace' keyword in pd.read_table is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_table(filepath, delim_whitespace=True)
/var/folders/5y/74w_zv855875xvjtmhp4g1xw0000gn/T/ipykernel_22347/1717233576.py:2: FutureWarning: The 'delim_whitespace' keyword in pd.read_table is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_table(filepath, delim_whitespace=True)
/var/folders/5y/74w_zv855875xvjtmhp4g1xw0000gn/T/ipykernel_22347/1717233576.py:2: FutureWarning: The 'delim_whitespace' keyword in pd.read_table is deprecated a

,time,WSPD


In [71]:
def to_str(dt):
    return dt.strftime('%m/%d/%Y')
len(np.unique(fin_df['time'].apply(to_str)))

0

In [72]:
len(np.unique(process_wind_file("Data/Wind/wind" + str(2013) + ".txt")['time']))

/var/folders/5y/74w_zv855875xvjtmhp4g1xw0000gn/T/ipykernel_22347/1717233576.py:2: FutureWarning: The 'delim_whitespace' keyword in pd.read_table is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_table(filepath, delim_whitespace=True)


0

In [73]:
process_wind_file("Data/Wind/wind" + str(2012) + ".txt")

/var/folders/5y/74w_zv855875xvjtmhp4g1xw0000gn/T/ipykernel_22347/1717233576.py:2: FutureWarning: The 'delim_whitespace' keyword in pd.read_table is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_table(filepath, delim_whitespace=True)


,time,WSPD
